In [ ]:
# 모델.py

In [1]:
import random
import math
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from transformers import AutoTokenizer, AutoModel
from transformers import get_linear_schedule_with_warmup
from transformers import get_cosine_schedule_with_warmup
from peft import LoraConfig, get_peft_model, TaskType #, AdapterConfig

In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)            # 기본 Python random 고정
    np.random.seed(seed)         # NumPy 랜덤 고정
    torch.manual_seed(seed)      # CPU 연산 랜덤 고정
    torch.cuda.manual_seed(seed) # GPU 모든 디바이스 랜덤 고정
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU일 때

    # 연산 재현성
    torch.backends.cudnn.deterministic = True  # cuDNN 연산을 determinisitc으로 강제
    torch.backends.cudnn.benchmark = False     # CUDA 성능 자동 튜닝 기능 끔 → 완전 재현 가능

set_seed(42)

In [3]:
EPOCHS = 20
WARMUP_RATIO = 0.1
LEARNING_RATE = 1e-3
BATCH_SIZE = 128
TEMPERATURE = 0.05
NEG_RATIO = 0.2

In [6]:
class E5LoRABackbone(nn.Module):
    def __init__(self, model_name: str, lora_cfg: dict):
        super().__init__()
        
        base_model = AutoModel.from_pretrained(model_name)

        # Linear(d → d) -→ Linear(d → d) + LoRA(d → d)
        lora_config = LoraConfig( 
            task_type=TaskType.FEATURE_EXTRACTION, # 임베딩 fine-tuning
            # LoRA가 분류기와 같은 output head에 적용되는 것이 아니라
            # 모델의 Transformer 블록(encoder)에만 적용되도록
            r=lora_cfg["r"],    # LoRA rank
            lora_alpha=lora_cfg["alpha"],
            lora_dropout=lora_cfg["dropout"],
            bias="none"
        )

        self.encoder = get_peft_model(base_model, lora_config)
        self.config = self.encoder.config # hidden_size 같은거 head에서 알아야함
    
    def forward(self, input_ids, attention_mask, **kwargs):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        # return outputs.last_hidden_state[:, 0] single vector 테스트할땐 일케 했는디..
        return outputs.last_hidden_state # head에 넣을거라 CLS pooling 안함

In [7]:
class SimpleMultiVectorHead(nn.Module):
    def __init__(self, num_vectors=3,  input_dim=384):
        super().__init__()

        # k개의 학습 가능한 쿼리 토큰 생성
        self.query_tokens = nn.Parameter( # 일단 텐서랑 다르게 학습 가능함
            torch.randn(1, num_vectors, input_dim) # (1, K, D) 차원의 랜덤값
        )
        # nn.init.normal_(self.query_tokens, std=0.02)
        # nn.init.orthogonal_(self.query_tokens)
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=8, batch_first=True)
        self.norm = nn.LayerNorm(input_dim)
    
    def forward(self, seq_out, attn_mask):
        """
        # shapes
        query = queries  # (B, K, D)
        key = seq_out    # (B, L, D)
        value = seq_out  # (B, L, D)
        """
        batch_size = seq_out.shape[0] # (B)
        queries = self.query_tokens.repeat(batch_size, 1, 1)  # Query 확장 (1, K, D) -> (B, K, D)
        key_padding_mask = ~attn_mask.bool()

        vectors, _ = self.attention( # (B, K, D)
            query=queries,
            key=seq_out,
            value=seq_out,
            key_padding_mask=key_padding_mask
        )

        # 2. 🔥 [Pro Tip] 잔차 연결 (Residual) + LayerNorm
        # "새로 배운 정보(vectors)에 원래 내 자아(queries)를 섞는다"
        # 이렇게 하면 학습이 훨씬 안정적으로 변합니다.
        # vectors = self.norm(queries + vectors)
        return vectors

In [8]:
class BookEmbeddingModel(nn.Module):
    def __init__(self, model_name: str, lora_config: dict):
        super().__init__()
        self.backbone = E5LoRABackbone(model_name, lora_config)
        self.head = SimpleMultiVectorHead(num_vectors=2, input_dim=self.backbone.config.hidden_size)
    
    def forward(self, input_ids, attention_mask, **kargs):
        sequence_output = self.backbone(input_ids, attention_mask) # (B, L, D)
        embeddings = self.head(sequence_output, attention_mask) # (B, k, D)
        return F.normalize(embeddings, p=2, dim=2) # contrastive loss 계산하려면 필수

In [10]:
model_name = "intfloat/e5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

teacher_model = AutoModel.from_pretrained(model_name)
teacher_model.eval()
for param in teacher_model.parameters():
    param.requires_grad = False 

lora = {'r': 16, 'alpha': 32, 'dropout': 0.1}
model = BookEmbeddingModel(model_name, lora)
device = "cuda" if torch.cuda.is_available() else "cpu"
teacher_model.to(device)
model.to(device)

BookEmbeddingModel(
  (backbone): E5LoRABackbone(
    (encoder): PeftModelForFeatureExtraction(
      (base_model): LoraModel(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 384, padding_idx=0)
            (position_embeddings): Embedding(512, 384)
            (token_type_embeddings): Embedding(2, 384)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0-11): 12 x BertLayer(
                (attention): BertAttention(
                  (self): BertSdpaSelfAttention(
                    (query): lora.Linear(
                      (base_layer): Linear(in_features=384, out_features=384, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.1, inplace=False)
                      )
               

In [11]:
# book_path = './data/e5_book_meta.parquet'
book_path = './test/book_meta.parquet'
books = pd.read_parquet(book_path)

In [12]:
def build_text(row): # 입력 텍스트 생성 (타이틀 + 설명 + 저자 등 결합)
    parts = [
        f"Title: {row['title']} |",
        # f"Category: {row['category']} |", # oracle
        f"Description: {row['description']}"
    ]
    return " ".join( # 리스트의 문자열들을 공백으로 연결할건데.....
        [p for p in parts if isinstance(p, str)] # NaN이나 None이 있으면 제외함
    ) # 최종적으로 하나의 문장 형태로 반환한다고 함!! "Title: ... Category: ... Description: ..."

books["text"] = books.apply(build_text, axis=1) # 새 컬럼 text에 대해서.... 문장 만듦

# 100개 미만인 카테고리는 노이즈로 간주하고 삭제
counts = books['category'].value_counts()
valid_categories = counts[counts > 100].index
books = books[books['category'].isin(valid_categories)]

In [13]:
dataset = Dataset.from_pandas(books)

le = LabelEncoder()
le.fit(dataset['category'])   # 전체 데이터로 학습

def encode_label(x):
    return {"label": le.transform([x["category"]])[0]}

dataset = dataset.map(encode_label)

num_classes = len(le.classes_)

Map:   0%|          | 0/81845 [00:00<?, ? examples/s]

collate_fn은 raw text와 label을 텐서로 묶어 모델이 학습할 수 있는 형태로 만들어줌
DataLoader는 이 함수로 미리 전처리한 batch를 모델에 공급하는 역할을 함
```
Dataset row(dict)
     ↓ (DataLoader)
batch = [row1, row2, ...] (list)
     ↓ (collate_fn)
텍스트 리스트 + 라벨 리스트
     ↓ (tokenizer)
input_ids, attention_mask (tensor)
     ↓
(inputs, labels)
     ↓
model(**inputs)
```

In [14]:
# Transformer 모델은 이런 raw 텍스트를 바로 처리 못 하고
# 토크나이저를 거쳐 tensor(batch_input_ids, batch_attention_mask) 형태가 필요함.
def collate_fn(batch): # DataLoader가 batch마다 호출
    # texts = [f"passage: {x['text']}" for x in batch]
    texts = [f"query: {x['text']}" for x in batch]
    labels = torch.tensor([x['label'] for x in batch])  # 라벨을 int 리스트 → torch.tensor 로 변환

    """
    토크나이저:
    텍스트를 token id로 변환 (input_ids), attention_mask 생성,
    batch의 최대 length에 맞춰 패딩, 출력 타입은 PyTorch tensor

    { 'input_ids': tensor([[101,  ... , 102], ...]),
      'attention_mask': tensor([[1,1,1,0,0...], ...) }
    """
    inputs = tokenizer(
      texts, padding=True, truncation=True, max_length=256, return_tensors="pt")

    return inputs, labels

In [15]:
total_len = len(dataset)
train_len = int(total_len * 0.8)
valid_len = total_len - train_len

train_dataset, valid_dataset = random_split(dataset, [train_len, valid_len])

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
)
valid_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
)

지금의 구조는 contrastive loss, genre/content KD, dynamic α, GradNorm, multi-vector head 등이 서로 얽혀 있어 LR 변화에 매우 민감함

- contrastive(genre) 비중이 큰 α=0.8 상태
- genre/content 양방향 KD 압력
- multi-vector head 정렬 전
- GradNorm 비율이 아직 수렴 안 됨
- α decay가 아직 적용 전(장르 비중 과다)

warmup을 너무 짧게(0\~2%) 주면 learning rate가 지나치게 빨리 상승해 초기 단계에서 gradient explosion이나 embedding 붕괴가 일어나고, 반대로 너무 길게(10\~20%) 주면 LR이 천천히 올라가는 동안 contrastive 쪽 표현 학습이 지연되고 embedding collapse 위험이 커져 최종 성능(MRR, top-1)까지 떨어짐

스케쥴러 자체도 바꿨는데, Linear decay는 warmup 이후에 LR이 직선으로 급격히 떨어짐. 이는 학습 후반부 KD alignment와 contrastive alignment의 미세 조정을 막아 학습이 사실상 멈추는 문제가 있음. 반면 Cosine 스케줄러는 warmup 이후 LR을 완만한 곡선 형태로 감소시키기 때문에, 초반에는 안정성을 제공하고, 중반에는 충분한 LR을 유지하며, 후반에도 작은 폭이지만 의미 있는 업데이트를 이어갈 수 있을 것임

In [16]:
total_steps = len(train_loader) * EPOCHS

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
# AdamW 옵티마이저로 LoRA 파라미터만 학습
# LoRA 덕분에 실제 업데이트되는 파라미터는 전체의 1% 정도

# scheduler = get_linear_schedule_with_warmup(
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(total_steps * 0.1), # WARMUP_RATIO), 원래 linear 일때 썻던 값은데 0.1이었거든? 0.05로 줄이래
    num_training_steps=total_steps,
)

In [17]:
import torch
import torch.nn.functional as F

def compute_retrieval_accuracy(model, dataloader, device, tokenizer, k=10):
    """
    Multi-Vector 모델용 검증 함수
    * Vector 0 (Genre Vector)만 사용하여 카테고리 검색 성능을 측정합니다.
    """
    embeddings_list = []
    labels_list = []

    model.eval() # 평가 모드 필수!

    with torch.no_grad():
        for batch_inputs, labels in dataloader:
            # 1. 입력 데이터 준비
            # input_ids = batch_inputs.to(device)
            batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()}
            labels = labels.to(device)

            # 2. 모델 Forward (Multi-Vector 출력)
            # output shape: (Batch, K, 768)
            multi_vectors = model(**batch_inputs)

            # 3. ★핵심★: Vector 0 (장르 벡터)만 추출
            # 우리가 검증하려는 건 "카테고리(장르)를 잘 맞추는가?" 이므로 0번만 씀
            genre_embeddings = multi_vectors[:, 0, :]
            genre_embeddings = F.normalize(genre_embeddings, p=2, dim=1)

            embeddings_list.append(genre_embeddings)
            labels_list.append(labels)

    # 전체 데이터 합치기
    all_embeddings = torch.cat(embeddings_list, dim=0)
    all_labels = torch.cat(labels_list, dim=0)
    similarity = torch.matmul(all_embeddings, all_embeddings.T) # (N, N) 행렬 계산
    similarity.fill_diagonal_(-1e9) # 자기 자신 제외 (-무한대 마스킹)

    _, topk_idx = similarity.topk(k, dim=1)  # top-k neighbor 인덱스
    nn_labels_topk = all_labels[topk_idx] # 이웃들의 라벨 가져오기 (N, k)

    # 정답 여부 행렬 (True/False)
    # 내 라벨(all_labels)과 이웃 라벨(nn_labels_topk)이 같은지 비교
    # unsqueeze(1)로 (N, 1) vs (N, k) 브로드캐스팅 비교
    hits = (nn_labels_topk == all_labels.unsqueeze(1)) # (N, k) Bool Tensor

    # -----------------------------
    # top-1: 가장 가까운 neighbor 정답 여부
    # top-k: k개 안에 정답이 하나라도 있으면
    # precision@k: k개 neighbor 중 정답 비율 평균
    # MRR: 정답이 rank 몇 번째인지에 따른 평균 역수 → rank 1이면 1.0, rank 2이면 0.5
    # -----------------------------
    # Top-1 accuracy
    top1_labels = nn_labels_topk[:, 0]
    top1_acc = (top1_labels == all_labels).float().mean().item()

    # Top-k accuracy (at least 1 match)
    topk_match = (nn_labels_topk == all_labels.unsqueeze(1)).any(dim=1).float()
    topk_acc = topk_match.mean().item()

    # Precision@k
    precision_at_k = (nn_labels_topk == all_labels.unsqueeze(1)).float().mean(dim=1).mean().item()

    # MRR (Mean Reciprocal Rank)
    ranks = (nn_labels_topk == all_labels.unsqueeze(1)).float() # 정답 label 위치 찾기
    reciprocal_rank = []
    for i in range(ranks.size(0)):
        pos_positions = torch.nonzero(ranks[i]).flatten()
        if len(pos_positions) == 0: # positive 없으면 reciprocal rank = 0
            reciprocal_rank.append(0.0)
        else:
            reciprocal_rank.append(1.0 / (pos_positions[0].item() + 1))
    mrr = sum(reciprocal_rank) / len(reciprocal_rank)

    metrics = {
        "top1_acc": top1_acc,
        "topk_acc": topk_acc,
        "precision@k": precision_at_k,
        "mrr": mrr
    }

    return metrics

In [18]:
def hard_negative(embeddings, labels, neg_ratio=0.1):
    batch_size = embeddings.size(0)
    k = max(3, int(batch_size * neg_ratio))
    similarity = torch.matmul(embeddings, embeddings.T)

    pos_mask = labels.unsqueeze(1) == labels.unsqueeze(0)

    sim_for_neg = similarity.clone()
    sim_for_neg.masked_fill_(pos_mask, -1e9)
    hard_neg_sims, _ = sim_for_neg.topk(k, dim=1) # (N, k)

    return hard_neg_sims

In [19]:
def noisy_contrastive_loss(embeddings, labels):

    similarity = torch.matmul(embeddings, embeddings.T)

    labels_eq = labels.unsqueeze(1) == labels.unsqueeze(0)
    identity_mask = torch.eye(len(labels), device=labels.device).bool() # 자기 자신 제거 mask
    labels_eq = labels_eq & (~identity_mask)

    pos_mask = labels_eq.float()   # (N, N)

    pos_sim = similarity * pos_mask
    neg_sim = hard_negative(embeddings, labels, neg_ratio=NEG_RATIO)

    pos_sim = pos_sim / TEMPERATURE
    neg_sim = neg_sim / TEMPERATURE

    loss = -torch.log(
        torch.exp(pos_sim).sum(dim=1) /
        (torch.exp(pos_sim).sum(dim=1) + torch.exp(neg_sim).sum(dim=1))
    ).mean()

    return loss

In [21]:
def calc_grad_norm(loss, model_layer):
    """
    특정 Loss가 특정 레이어(model_layer)의 파라미터에 가하는
    Gradient의 총량(Norm)을 계산합니다.
    """
    # 1. 해당 레이어의 파라미터만 가져옴 (requires_grad=True인 것만)
    params = [p for p in model_layer.parameters() if p.requires_grad]

    if not params:
        return 0.0

    # 2. Gradient 계산 (create_graph=False, retain_graph=True 필수!)
    # retain_graph=True: 뒤에 진짜 backward()를 또 해야 하므로 그래프를 날리면 안 됨
    grads = torch.autograd.grad(
        loss,
        params,
        retain_graph=True,
        allow_unused=True
    )

    # 3. Norm(크기) 합산 (L2 Norm)
    total_norm = 0.0
    for g in grads:
        if g is not None:
            total_norm += g.pow(2).sum().item()

    return total_norm ** 0.5

In [22]:
t = 200  # 충돌 시작 지점
base_alpha = 0.8   # 초반: 장르(Genre) 정보를 확실히 잡음
target_alpha = 0.2 # 후반: 충돌 회피를 위해 장르 비중을 낮춤 (본문 집중)
steps_per_epoch = len(train_loader)
running_ratio = 1.0
beta = 0.95  # 관성 계수 (클수록 변화가 부드러움)

for epoch in range(EPOCHS):
    model.train()
    total_train_loss = 0

    for step, (batch_inputs, labels) in enumerate(tqdm(train_loader, desc = f"Epoch: {epoch+1}")):
        global_step = epoch * steps_per_epoch + step

        if global_step < t:
            alpha = base_alpha
        else:
            x = 5 * (global_step - t) / t
            sigmoid_x = 1 / (1 + math.exp(-x))

            decay_ratio = (sigmoid_x - 0.5) * 2
            if decay_ratio > 1.0: decay_ratio = 1.0

            alpha = base_alpha - (base_alpha - target_alpha) * decay_ratio

        batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()}
        labels = labels.to(device)

        student_vectors = model(**batch_inputs)
        
        genre_vector = student_vectors[:, 0, :]
        content_vectors = student_vectors[:, 1, :]
        
        loss_cont = noisy_contrastive_loss(genre_vector, labels)
    
        with torch.no_grad():
            teacher_outputs = teacher_model(**batch_inputs)
            hidden = teacher_outputs.last_hidden_state       # (B, L, D)
            mask = batch_inputs['attention_mask'].unsqueeze(-1)  # (B, L, 1)
            teacher_embeddings = (hidden * mask).sum(dim=1) / mask.sum(dim=1)
            teacher_norm = F.normalize(teacher_embeddings, p=2, dim=1)

        genre_norm = F.normalize(genre_vector, p=2, dim=1)
        loss_kd_genre = F.mse_loss(genre_norm, teacher_norm)
        
        content_norm = F.normalize(content_vector, p=2, dim=1)
        loss_kd_content = F.mse_loss(content_norm, teacher_norm)
        
        loss_kd_combined = alpha * loss_kd_genre + (1 - alpha) * loss_kd_content

        norm_main = calc_grad_norm(loss_cont, model.head.attention)
        norm_sub = calc_grad_norm(loss_kd_combined, model.head.attention)
        target_scale = 0.6
        current_ratio = norm_main / (norm_sub + 1e-8) * target_scale
        if current_ratio > 1000.0: current_ratio = 1000.0
        running_ratio = beta * running_ratio + (1 - beta) * current_ratio

        total_loss = loss_cont + running_ratio * loss_kd_combined

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        scheduler.step()

        total_train_loss += total_loss.item()

    train_loss = total_train_loss / len(train_loader)

    model.eval()
    metrics = compute_retrieval_accuracy(model, valid_loader, device, tokenizer)
    print(f"[Epoch {epoch + 1}] Train Loss: {train_loss:.4f}")
    print(f"Top-1 Accuracy : {metrics['top1_acc']:.4f} | Top-10 Accuracy : {metrics['topk_acc']:.4f}")
    print(f"Precision@10   : {metrics['precision@k']:.4f} | MRR              : {metrics['mrr']:.4f}")

Epoch: 1: 100%|██████████████████████████████████████████████████████████████████| 512/512 [02:24<00:00,  3.56it/s]


[Epoch 1] Train Loss: 2.5856
Top-1 Accuracy : 0.5741 | Top-10 Accuracy : 0.8661
Precision@10   : 0.5402 | MRR              : 0.6755


Epoch: 2: 100%|██████████████████████████████████████████████████████████████████| 512/512 [02:23<00:00,  3.57it/s]


[Epoch 2] Train Loss: 2.4089
Top-1 Accuracy : 0.5984 | Top-10 Accuracy : 0.8666
Precision@10   : 0.5643 | MRR              : 0.6918


Epoch: 3: 100%|██████████████████████████████████████████████████████████████████| 512/512 [02:23<00:00,  3.57it/s]


...

[Epoch 6] Train Loss: 2.0043
Top-1 Accuracy : 0.6127 | Top-10 Accuracy : 0.8570
Precision@10   : 0.5891 | MRR              : 0.7000


Epoch: 7: 100%|██████████████████████████████████████████████████████████████████| 512/512 [02:23<00:00,  3.56it/s]


[Epoch 7] Train Loss: 1.9436
Top-1 Accuracy : 0.6137 | Top-10 Accuracy : 0.8553
Precision@10   : 0.5962 | MRR              : 0.6998


Epoch: 8:   9%|██████                                                             | 46/512 [00:13<02:13,  3.48it/s]


KeyboardInterrupt: 

In [ ]:
import os
save_path = f"./old_multi_vec_{LAMBDA}_20ep.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)